In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import threading
import joblib
import numpy as np
from scipy.sparse import hstack

# Load components
model = joblib.load('svm_model_best.pkl')
vectorizer = joblib.load('vectorizer_best.pkl')
scaler = joblib.load('scaler_best.pkl')

# Colors and fonts
BG_COLOR = "#0f1117"
TEXT_COLOR = "#e0e0e0"
INPUT_BG = "#1e1e1e"
INPUT_BORDER = "#3c3c3c"
ACCENT_BLUE = "#40c4ff"
FAKE_COLOR = "#ff3d00"
GENUINE_COLOR = "#00e676"
FONT = ("Segoe UI", 11)
TITLE_FONT = ("Segoe UI", 20, "bold")
BANNER_FONT = ("Segoe UI", 14, "bold")

# Main window
root = tk.Tk()
root.title("Review Authenticity Detector")
root.geometry("800x600")
root.configure(bg=BG_COLOR)

# Style
style = ttk.Style()
style.theme_use("clam")
style.configure("TLabel", background=BG_COLOR, foreground=TEXT_COLOR, font=FONT)
style.configure("TEntry", foreground=TEXT_COLOR, fieldbackground=INPUT_BG, background=INPUT_BG, relief="flat", padding=5)
style.configure("TButton", font=("Segoe UI", 11, "bold"), padding=6, relief="flat",
                background=INPUT_BG, foreground=TEXT_COLOR)
style.map("TButton",
          background=[("active", ACCENT_BLUE)],
          foreground=[("active", "#ffffff")])

# Title
ttk.Label(root, text="🧠 Fake vs Genuine Review Detector", font=TITLE_FONT, foreground=ACCENT_BLUE).pack(pady=20)

# --- Review Text Label and Box ---
review_container = tk.Frame(root, bg=BG_COLOR)
review_container.pack(padx=40, anchor="center")

# Label outside the box, aligned top-left
review_label = tk.Label(review_container, text="Review Text:", font=("Segoe UI", 11, "bold"),
                        bg=BG_COLOR, fg=ACCENT_BLUE)
review_label.pack(anchor="w", pady=(0, 4))

# Centered text box below label
review_entry = tk.Text(review_container, height=6, width=80, font=FONT,
                       bg=INPUT_BG, fg=TEXT_COLOR, insertbackground=TEXT_COLOR,
                       relief="flat", highlightthickness=1, highlightbackground=INPUT_BORDER)
review_entry.pack()

# --- Metadata Inputs ---
meta_frame = tk.Frame(root, bg=BG_COLOR)
meta_frame.pack(pady=5)

def create_input(label_text):
    frame = tk.Frame(meta_frame, bg=BG_COLOR)
    frame.pack(anchor="w", padx=40, pady=6)
    ttk.Label(frame, text=label_text, width=20).pack(side=tk.LEFT)
    entry = ttk.Entry(frame, width=25)
    entry.pack(side=tk.LEFT)
    entry.configure(style="TEntry")
    return entry

reviewwordcount_entry = create_input("Review Word Count:")
helpfulnessratio_entry = create_input("Helpfulness Ratio:")
userreviewcount_entry = create_input("User Review Count:")

# Loading label
loading_label = ttk.Label(root, text="", foreground=ACCENT_BLUE, font=FONT)
loading_label.pack(pady=10)

# Result banner frame
result_banner = tk.Frame(root, bg=BG_COLOR, pady=10)
result_banner.pack()

result_label = tk.Label(result_banner, text="", font=BANNER_FONT, width=45, pady=10,
                        relief="solid", bd=2)
result_label.pack()

# Loading animation
def show_loading():
    loading_label.config(text="Analyzing")
    for _ in range(4):
        loading_label.config(text=loading_label.cget("text") + ".")
        root.update()
        root.after(120)

# Main logic
def analyze():
    review_text = review_entry.get("1.0", tk.END).strip()
    if not review_text:
        messagebox.showerror("Input Error", "Please enter review text.")
        return

    try:
        rwc = float(reviewwordcount_entry.get())
        hr = float(helpfulnessratio_entry.get())
        urc = float(userreviewcount_entry.get())
    except ValueError:
        messagebox.showerror("Input Error", "Enter valid numbers for all metadata fields.")
        return

    result_label.config(text="", bg=BG_COLOR)
    show_loading()

    tfidf_features = vectorizer.transform([review_text])
    meta_features = np.array([[rwc, hr, urc]])
    scaled_meta = scaler.transform(meta_features)
    combined = hstack([tfidf_features, scaled_meta])

    prediction = model.predict(combined)[0]

    if prediction == 1:
        result_label.config(text="✔ GENUINE REVIEW DETECTED", bg=GENUINE_COLOR, fg="#000000")
    else:
        result_label.config(text="✘ FAKE REVIEW DETECTED", bg=FAKE_COLOR, fg="#ffffff")

    loading_label.config(text="")

def run_thread():
    threading.Thread(target=analyze).start()

# Predict Button
ttk.Button(root, text="Detect Review", command=run_thread).pack(pady=20)

root.mainloop()




/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
